In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('lending-club-data.csv')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,sub_grade_num,delinq_2yrs_zero,pub_rec_zero,collections_12_mths_zero,short_emp,payment_inc_ratio,final_d,last_delinq_none,last_record_none,last_major_derog_none
0,1077501,1296599,5000,5000,4975,36 months,10.65,162.87,B,B2,...,0.4,1.0,1.0,1.0,0,8.14350,20141201T000000,1,1,1
1,1077430,1314167,2500,2500,2500,60 months,15.27,59.83,C,C4,...,0.8,1.0,1.0,1.0,1,2.39320,20161201T000000,1,1,1
2,1077175,1313524,2400,2400,2400,36 months,15.96,84.33,C,C5,...,1.0,1.0,1.0,1.0,0,8.25955,20141201T000000,1,1,1
3,1076863,1277178,10000,10000,10000,36 months,13.49,339.31,C,C1,...,0.2,1.0,1.0,1.0,0,8.27585,20141201T000000,0,1,1
4,1075269,1311441,5000,5000,5000,36 months,7.90,156.46,A,A4,...,0.8,1.0,1.0,1.0,0,5.21533,20141201T000000,1,1,1


## create target column

In [4]:
data['safe_loans'] = data['bad_loans'].apply( lambda x: +1 if x==0 else -1 )
data.drop('bad_loans',axis=1,inplace=True)

In [6]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'

# Extract the feature columns and target column
data = data[features + [target]]

In [7]:
with open('module-5-assignment-2-train-idx.json','r') as f:
    train_idx = json.load(f)
with open('module-5-assignment-2-test-idx.json','r') as f:
    test_idx = json.load(f)

In [8]:
train = data.iloc[train_idx]
test = data.iloc[test_idx]
print(train.shape)
print(test.shape)

(37224, 5)
(9284, 5)


## One hot encoding

In [9]:
df = pd.get_dummies(data)
train_ = pd.get_dummies(train)
test_ = pd.get_dummies(test)

In [11]:
train_.shape

(37224, 25)

## Decision tree

In [15]:
## find mistakes
def intermediate_node_num_mistakes(label):
    if len(label) == 0:
        return 0    
    
    pos = (label==+1).sum()   
    
    neg = (label==-1).sum()               
   
    mistake = 0
    if(pos>neg):
        return neg
    else:
        return pos

In [16]:
def best_splitting_feature(data, features, target):
    
    target_values = data[target]
    best_feature = None 
    best_error = 10     
    
    num_data_points = float(len(data))  
    
    for feature in features:
        
        left_split = data[data[feature] == 0]
        
        right_split =  data[data[feature] == 1]
            
        left_mistakes = intermediate_node_num_mistakes(left_split[target])        

        right_mistakes = intermediate_node_num_mistakes(right_split[target])
            
        error = (left_mistakes + right_mistakes)/num_data_points

        if error < best_error:
            best_error = error
            best_feature = feature
    
    return best_feature 

    { 
       'is_leaf'            : True/False.
       'prediction'         : Prediction at the leaf node.
       'left'               : (dictionary corresponding to the left tree).
       'right'              : (dictionary corresponding to the right tree).
       'splitting_feature'  : The feature that this node splits on
    }

In [23]:
def create_leaf(target_values):    

    leaf = {'splitting_feature' : None,
            'left' : None,
            'right' : None,
            'is_leaf': True    }
   
    pos = len(target_values[target_values == +1])
    neg = len(target_values[target_values == -1])    

    if pos > neg:
        leaf['prediction'] = 1    
    else:
        leaf['prediction'] = -1           

    return leaf 

In [92]:
def decision_tree_create(data, features, target, current_depth = 0, max_depth = 10):
    
    remaining_features = features[:]
    
    target_values = data[target]
    
    print("--------------------------------------------------------------------")
    print("Subtree, depth = %s (%s data points)." % (current_depth, len(target_values)))
    

    # Stopping condition 1
    
    mistakes = intermediate_node_num_mistakes(target_values)
    if  mistakes == 0:
        print("Stopping condition 1 reached.")
        return create_leaf(target_values)
    
    # Stopping condition 2
    
    if remaining_features == []:
        print("Stopping condition 2 reached.")
        return create_leaf(target_values)    
    
    # Additional stopping condition
    
    if current_depth >= max_depth: 
        print("Reached maximum depth. Stopping for now.")
        return create_leaf(target_values)

    # Find the best splitting feature 
    
    best_feature = best_splitting_feature(data,remaining_features,target)

    # Split on the best feature that we found. 
    
    left_split = data[data[best_feature] == 0]
    right_split = data[data[best_feature]==1]
    
    remaining_features.remove(best_feature)
    
    print("Split on feature %s. (%s, %s)" % (best_feature, len(left_split), len(right_split)))
    
    # Create a leaf node if the split is "perfect"
    
    if len(left_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(left_split[target])
    
    if len(right_split) == len(data):
        print("Creating leaf node.")
        return create_leaf(right_split[target])

        
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    right_tree = decision_tree_create(right_split,remaining_features,target,current_depth + 1, max_depth)

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': best_feature,
            'left'             : left_tree, 
            'right'            : right_tree}

### Build tree

In [41]:
curr_depth = 0
max_depth = 6
features = list(train_.columns)
features.remove(target)
print(len(features))
my_tree = decision_tree_create(train_,features,target,curr_depth,max_depth)

24
--------------------------------------------------------------------
Subtree, depth = 0 (37224 data points).
Split on feature term_ 36 months. (9223, 28001)
--------------------------------------------------------------------
Subtree, depth = 1 (9223 data points).
Split on feature grade_A. (9122, 101)
--------------------------------------------------------------------
Subtree, depth = 2 (9122 data points).
Split on feature grade_B. (8074, 1048)
--------------------------------------------------------------------
Subtree, depth = 3 (8074 data points).
Split on feature grade_C. (5884, 2190)
--------------------------------------------------------------------
Subtree, depth = 4 (5884 data points).
Split on feature grade_D. (3826, 2058)
--------------------------------------------------------------------
Subtree, depth = 5 (3826 data points).
Split on feature grade_E. (1693, 2133)
--------------------------------------------------------------------
Subtree, depth = 6 (1693 data points)

In [44]:
my_tree

{'is_leaf': False,
 'prediction': None,
 'splitting_feature': 'term_ 36 months',
 'left': {'is_leaf': False,
  'prediction': None,
  'splitting_feature': 'grade_A',
  'left': {'is_leaf': False,
   'prediction': None,
   'splitting_feature': 'grade_B',
   'left': {'is_leaf': False,
    'prediction': None,
    'splitting_feature': 'grade_C',
    'left': {'is_leaf': False,
     'prediction': None,
     'splitting_feature': 'grade_D',
     'left': {'is_leaf': False,
      'prediction': None,
      'splitting_feature': 'grade_E',
      'left': {'splitting_feature': None,
       'left': None,
       'right': None,
       'is_leaf': True,
       'prediction': -1},
      'right': {'splitting_feature': None,
       'left': None,
       'right': None,
       'is_leaf': True,
       'prediction': -1}},
     'right': {'splitting_feature': None,
      'left': None,
      'right': None,
      'is_leaf': True,
      'prediction': -1}},
    'right': {'splitting_feature': None,
     'left': None,
     

## Predictions with decision tree

In [47]:
def classify(tree, x, annotate = False):
    
    if(tree['is_leaf']):
        if(annotate):
             print("At leaf, predicting %s" % tree['prediction'])
        return tree['prediction']
    
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        
        if(annotate):
             print("Split on %s = %s" % (tree['splitting_feature'], split_feature_value))
                
        if(split_feature_value == 0):
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'],x,annotate)

In [53]:
print(test_.iloc[0][target])
print('Predicted class: %s ' % classify(my_tree, test_.iloc[0]))

-1
Predicted class: -1 


In [55]:
classify(my_tree, test_.iloc[0], annotate=True)

Split on term_ 36 months = 0
Split on grade_A = 0
Split on grade_B = 0
Split on grade_C = 0
Split on grade_D = 1
At leaf, predicting -1


-1

In [57]:
test_.iloc[[0]]

,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,term_ 36 months,term_ 60 months,...,emp_length_10+ years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,emp_length_< 1 year
24,-1,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0


## Evaluate decision tree

In [68]:
def evaluate_classification_error(tree, data):
    
    prediction = []
    for i in range(len(data)):
        pred = classify(tree,data.iloc[i])
        prediction.append(pred)
    
    mistakes = (prediction!=data[target]).sum()
    error = mistakes/ float(len(data))
    
    return error

In [69]:
evaluate_classification_error(my_tree, test_)

0.37774666092201636

## Print decision stump

In [81]:
def print_stump(tree, name='root'):
    
    split_name = tree['splitting_feature'] 

    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    
    print('                       %s' % name                )
    print('         |---------------|----------------|'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name) )
    print('         |                                |'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('    (%s)                         (%s)' % 
          (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')) )

In [82]:
print_stump(my_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [83]:
print_stump(my_tree['left'], my_tree['splitting_feature'])

                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_A == 0]               [grade_A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [84]:
print_stump(my_tree['left']['left'], my_tree['left']['splitting_feature'])

                       grade_A
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_B == 0]               [grade_B == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)


In [85]:
print_stump(my_tree['left']['left']['left'], my_tree['left']['left']['splitting_feature'])

                       grade_B
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_C == 0]               [grade_C == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)


In [90]:
def print_stump_(tree, name='root'):
    
    split_name = tree['splitting_feature'] 

    if split_name is None:
        print("(leaf, label: %s)" % tree['prediction'])
        return None
    
    print('                       %s' % name                )
    print('         |---------------|----------------|'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('  [{0} == 0]               [{0} == 1]    '.format(split_name) )
    print('         |                                |'     )
    print('         |                                |'     )
    print('         |                                |'     )
    print('    (%s)                         (%s)' % 
          (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree')) )
    
    #print_stump_(tree['left'],tree['splitting_feature'])
    print_stump_(tree['right'],tree['splitting_feature'])

In [91]:
print_stump_(my_tree)

                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term_ 36 months == 0]               [term_ 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
                       term_ 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade_D == 0]               [grade_D == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (leaf, label: -1)
(leaf, label: -1)
